# Lecture 9

Lecture 9 will cover operator overloading and iterators.

Reference
 * [2] Section 9.8-9.10
 
Other reading:
 * https://www.geeksforgeeks.org/operator-overloading-in-python/
 * https://realpython.com/operator-function-overloading/
 * https://www.tutorialsteacher.com/python/magic-methods-in-python 
 * https://www.w3schools.com/python/python_iterators.asp
 * https://www.programiz.com/python-programming/iterator

# Operator overloading special methods

You have seen a few overloaded operators, but there are several important methods.

They are all named according to `__name__` convention (two underscores around the name). These methods are sometimes called "magic methods" or "dunder methods".

List of a bunch of magic methods: https://rszalski.github.io/magicmethods/

Some (possibly useful) *unary* (only taking one argument) operators:
* repr: `repr(x)` (data **repr**esentation)
* len: `len(x)`
* abs: `abs(x)`
* neg: `-a`

and many other things


Common conversions:
* str
* int
* float
* complex
* bool

Lets test out a bunch of these:

In [ ]:
class MyClass:
    def __init__(self, x):
        self.x = x

    # "truthyness"
    def __bool__(self):
        return self.x > 0

    def __len__(self):
        return self.x

    def __neg__(self):
        return MyClass(-self.x)

    def __str__(self):
        return "x = " + str(self.x)
    
    def __repr__(self):
        return "MyClass(" + repr(self.x) + ")"

In [ ]:
x = MyClass(3)
print(f"{x} has length {len(x)}")
if x:
    print("x is truthy")
print([x, -x])
y = -x
if y:
    print("y is truthy")
else:
    print("y is NOT truthy")

In [ ]:
num = 0
if num:
    print("It was truthy")

### Binary operators

There are a massive amount of binary operators (two argument functions) one can overload: `+ * - / // | & << **`.

Infix operator: Operator goes between the operands, for example `a * b`.

If is often in the case that both sides of the infix operator has the same type, and the output from the computation is also of the same type, but this is not strictly necessary.

Python will call the suitable special method by doing `a + b === a.__add__(b)`, with similar names for the multitude of other operators (`mul`, `div`, etc.), If `a.__add__` isn't found, then the corresponding "reverse" operation is used: `b.__radd__(a)` (if available).

Many operators have extended assignments as well, and these can be accessed through special methods as well, e.g.
`a += b` which becomes `a = a.__iadd__(b)`.

In [ ]:
l = [1,2,3]
l2 = l
l = l + [4,5]
print(l)
print(l2)

In [ ]:
import math

class Fraction:
    
    @staticmethod
    def _normalize(num, denom):
        d = math.gcd(num, denom)
        return num // d, denom // d

    def __init__(self, num: int, denom: int):
        self.num, self.denom = self._normalize(num, denom)

    def __str__(self):
        return str(self.num) + "/" + str(self.denom)
    
    def __float__(self):
        return self.num / self.denom

    def __add__(self, other: Fraction):
        # Expecting "other" to be a Fraction.
        # The result is a Fraction as well.
        return Fraction(self.num * other.denom + other.num * self.denom,
                        self.denom * other.denom)
    
    def __iadd__(self, other: Fraction):
        # self += other
        # We want to modify "self"!
        # Note: mutating conceptually immutable things (like numbers) can
        # make the code confusing
        num = self.num * other.denom + other.num * self.denom
        denom = self.denom * other.denom
        self.num, self.denom = self._normalize(num, denom)
        return self
        
    def __mul__(self, other: int):
        return Fraction(self.num * other.num, self.denom * other.denom)

    def __pow__(self, exponent: int):
        # This assumes that the exponent is an integer
        return Fraction(self.num ** exponent, self.denom ** exponent)

In [ ]:
x = Fraction(3, 4)
y = Fraction(2, 3)
x2 = x

print(f"x = {x}")
print(f"x2 = {x2}")
print(f"y = {y}")


q = y ** 2
x += q
z = x + y
w = x * y

print("----")
print(f"x = {x}")
print(f"x2 = {x2} !!!!!") # Note!
print(f"float(z) = {float(z)}")
print(f"q = {q}")
print(f"w = {w}")

**Questions**:
* In Python, will `a + b` always give the same result as `b + a`? 
* How about `a * b` and `b * a`?
* Will `a += b` always be the same as `a = a + b`?'

### Comparison operators

Comparison operators have a few special rules, because they have no "reversed" operations.
Instead, `a < b` is used like `b > a` when `>`-operation is missing.

Implementing at least one comparison operator from `__lt__`, `__gt__` or `__cmp__` is necessary for comparisons (sort will try them in that order). Typically, `__lt__` is implemented.

In [ ]:
class MyTestClass:
    """ A test class where only one instance 
    variable determines the comparison operators
    """
    
    def __init__(self, val, val2):
        self.val = val
        self.val2 = val2

    def __eq__(self, other):
        return abs(self.val) == abs(other.val)

    def __lt__(self, other): # We only check the magnitude:
        return abs(self.val) < abs(other.val)


In [ ]:
x = MyTestClass(-34, "Hello")
y = MyTestClass(34, "World")
print(x > y) # Python will look for a match, and will call y < x (which should be mathematically equivalent)
print(x == y)

In [ ]:
x = MyTestClass(34, "Hello")
y = MyTestClass(-40, "World")
l = [x, y, y, x]
l.sort()
print(l)

If `__eq__` is not implemented, the `is` operator is used instead.

In [ ]:
class MyTestClass():
    def __init__(self, val):
        self.val = val

    def __lt__(self, other): # We only check the magnitude:
        return abs(self.val) < abs(other.val)

In [ ]:
x = MyTestClass(23)
y = MyTestClass(23)
print(x == y)
print(x == x)

In [ ]:
x

In [ ]:
y

This strange default behavior is rarely what we want! Therefore, `__eq__` is recommended to overload if `__lt__` has been overloaded.

## Indexing overloading

In [ ]:
class LoggingList(list):
    def __setitem__(self, pos, val):
        print(f"Setting items {val} at: {pos}")
        super().__setitem__(pos, val)
        
    def __getitem__(self, pos):
        print(f"Getting items at: {pos}")
        return super().__getitem__(pos)

    def __delitem__(self, pos):
        print(f"Deleting items at: {pos}")
        return super().__delitem__(pos)

In [ ]:
x = LoggingList()
x += [1,2,3,4] #      __iadd__ is called

x[0:2] = [123, 456] # __setitem__ is called
y = x[2:] #           __getitem__ is called
print(y)
del x[0] #            __delitem__ is called
print(x)

Note that `a:b:c` is just shorthand for creating a `slice` object.
While this is the most common use for the  `__***item__` methods, this is not at all required, e.g:

In [ ]:
class IdentityMatrix:
    def __getitem__(self, row_and_col):
        print(f"Getting items at: {row_and_col}")
        row, col = row_and_col
        return 1 if row == col else 0

eye = IdentityMatrix()
print(eye[4,5], eye[7,7])

In [ ]:
class CaseInsensitiveDataBase:
    def __init__(self):
        # Some made up data for this example:
        self.dataset = {'A': 7, 'B': 4, 'C': 7, 'D': 0}

    def __getitem__(self, entries):
        return [self.dataset[entry.upper()] for entry in entries]

db = CaseInsensitiveDataBase()
print(db['b', 'd'])

## Comment on methods vs operators

This kind of compact notation is often sought by programmers, why have
```python
x.add(y)
```
when you could have
```python
x + y
```

But it's often the case that we are not working with mathematics, but strings, lists, graphical objects, and other types of custom objects.
Therefore, it's often not clear what the operator will actually do (languages like Java disallow this operator overloading for this reason).

When comparing two Car-objects, do we just check if they are of the same model? Same color? Model and color? Same registration number? Top speed?

Operators should not be overused, since it is often more clear to write a descriptive method name. Use it only when the purpose is clear.

Operator overloading:
```python
elf = Elf()
dwarf = Dwarf()
elf += Bow()
dwarf += Axe()
elf >> frodo
dwarf >> frodo
```

Well named methods:
```python
elf = Elf()
dwarf = Dwarf()
elf.add_weapon(Bow())
dwarf.add_weapon(Axe())
elf.pledge_alligence(frodo)
dwarf.pledge_alligence(frodo)
```

### Exercise 

(maybe too big to do during lecture but you can start):

- Create a class called `Polynomial`.
- Create a constructor that takes coefficients for the polynomial, e.g: `Polynomial(1,2,2)` represents `x^2 + 2x + 2`.
- Create a `__str__` method that prints it nicely.
- Overload `__add__`, `__neg__`, `__mul__` etc.
- Create the `derivative` method returning a new `Polynomial`.
- Overload `__call__` to evaluate the polynomial for some `x`.

Example usage:

```python
>>> p = Polynomial(1,2,2)

>>> print(p)
x^2 + 2x + 2

>>> print(p + Polynomial(3,2,1,2))
x^3 + 3x^2 + 3x + 4

>>> print(p.derivative())
2x + x

>>> p(2.0)
10.0
```

# Iterators and generators

In the course so far, we've used multiple examples of the for-loop:

In [ ]:
for i in range(5):
    print(i, end=",")

In [ ]:
for x in "Hello World":
    print(x, end=", ")

In [ ]:
for x in zip([6,3,4], "ABC"):
    print(x, end=", ")

All these examples are very different. The `range` keeps generating new numbers until the max value is reached. In the string, we go letter by letter until the end of the string.
The `for`-loop is compact, convenient, and easy to read.

### The `for`-loop

Lets assume we have a loop like:
```python
for a in b:
    ...
```
When the `for`-loop starts, it asks for
```python
it = iter(b)    # Looks for b.__iter__()
```
and then it asks for
```python
a = next(it)    # Looks for it.__next__()
```
until a `StopIteration`-exception is raised.

Lets try it out, step-by-step

In [ ]:
b = "Hello"
it = iter(b)
print(it)

In [ ]:
print( next(it) )
print( next(it) )
print( next(it) )
print( next(it) )
print( next(it) )

So far so good, what happens when we ask one more time?

In [ ]:
next(it)

**To sum it up:** 
1. The object needs support the function `iter`
2. Whatever it returned from `iter` must support the function `next`

### Lets create a custom iterator

In [ ]:
# https://en.wikipedia.org/wiki/Collatz_conjecture
class CollatzGenerator:
    def __init__(self, n):
        self.n = n

    def __iter__(self):
        # We'll reuse the class generator itself 
        # as the iterator for this simple case
        return self
    
    def __next__(self):
        if self.n == 1:
            raise StopIteration
        if self.n % 2 == 0:
            self.n //= 2
        else:
            self.n = self.n*3 + 1
        
        return self.n

In [ ]:
# This should produce 3, 10, 5, 16, 8, 4, 2, 1.
x = CollatzGenerator(6)
for n in x:
    print(n, end=", ")

Note! In this `CollatzGenerator`, the iterator is the same as the generator itself.
If we try to use `x` again, it will already have reached 1, as the (modified) iterator *is* the generator.

In [ ]:
print(x.n)

In [ ]:
for n in x:
    print(n, end=", ")

In [ ]:
r = range(10)

In [ ]:
for i in r:
    print(n, end=", ")

In [ ]:
for i in r:
    print(n, end=", ")

### Another example, the infinite loop!

Lets create a loop over all natural numbers. We would like to have
```python
nat = NaturalNumbers():
for x in nat:
   ...
```
Of course, it's not very good that we modified the starting point of the `nat` (in case we want to reuse it).

We should therefor avoid `def __iter__(self): return self`, and instead return a different iterator.

In [ ]:
class NaturalNumbers:
    def __iter__(self):
        # Lets declare an iterator internally:
        class NaturalNumbers_It:
            def __init__(self):
                self.n = 0
       
            def __next__(self):
                self.n += 1
                return self.n
        # Each time we start a for-loop, 
        # we'll return a brand new iterator:
        return NaturalNumbers_It()

In [ ]:
nat = NaturalNumbers()

In [ ]:
tot = 0
for x in nat:
    tot += x
    # We might have some condition to stop that 
    # doesn't depend on only one the value of x!
    if tot > 1000:
        break
print(tot, x)

## Generators

The interested student can also look up the `yield` statement. For example here:
http://www.python-course.eu/python3_generators.php

The for-loop will obtain each `yield`'ed value, and the function will be frozen until the next yielded value is requested. This type of functions are often called "coroutines":

In [ ]:
def countfromto(a, b):
    while a < b:
        print("Yielding a new value!")
        yield a
        a += 1

for i in countfromto(10, 15):
    print(i)

In [ ]:
v = countfromto(10, 15)
print(next(v))
print(next(v))
print(next(v))
print(next(v))
print(next(v))
print(next(v))

Can make writing iterators more terse:

In [ ]:
# https://en.wikipedia.org/wiki/Collatz_conjecture
def collatz(n):
    while n != 1:
        yield n
        if n % 2 == 0:
            n //= 2
        else:
            n = n*3 + 1
    yield(n)

In [ ]:
list(collatz(25))

### Exercise 

- Create an iterator that iterates the prime numbers.
- Now do the same using a generator
- Create an iterator `taketwo` that grabs 2 elements from another iterator at once, returning a tuple

```python
for a, b in taketwo(range(6)):
    print(a, b)  # prints 0,1 then 2,3 then 4,5
```